In [2]:
from __future__ import print_function

import os

import matplotlib.pyplot as plt
import scipy.io as scio
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

from architecture import MST

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
# dtype = torch.cuda.DoubleTensor
# dtype = torch.double
device = torch.device('cuda')
torch.cuda.manual_seed(seed=666)


########################################### 辅助函数 ###########################################
def thres_21(L, tau, M, N, B):
    S = torch.sqrt(torch.sum(torch.mul(L, L), 2))
    S[S == 0] = 1
    T = 1 - tau / S
    T[T < 0] = 0
    R = T.reshape(M, N, 1).repeat((1, 1, B))
    res = torch.mul(R, L)
    return res


################################ 读取数据 ################################
Data = scio.loadmat("./Data.mat")
Clean, Noisy, Mask = Data["Clean"], Data["Cloud_image"], Data["Mask"]
Cloud = 1 - Mask
# Data = scio.loadmat("./SimuData.mat")
# Clean, Mask, Cloud, Noisy = Data["Clean"], Data["Mask"], Data["Cloud"], Data["Noisy"]

# M, N, = 384, 384
# M, N, = 256, 256
M, N = 400, 400
B, T = 4, 4
# 相邻四张图像为同一个时间，因而初始的维度为M, N, T, B
Clean = torch.from_numpy(Clean[:M, :N, :].reshape(M, N, T, B)).to(device)
Mask = torch.from_numpy(Mask[:M, :N, :].reshape(M, N, T, B)).to(device)  # 云=0
Cloud = torch.from_numpy(Cloud[:M, :N, :].reshape(M, N, T, B)).to(device)  # 云=1
Noisy = torch.from_numpy(Noisy[:M, :N, :].reshape(M, N, T, B)).to(device)


# 进行维度调整：M, N, B, T
# Clean = torch.permute(Clean, [0, 1, 3, 2])
# Mask = torch.permute(Mask, [0, 1, 3, 2])
# Cloud = torch.permute(Cloud, [0, 1, 3, 2])
# Noisy = torch.permute(Noisy, [0, 1, 3, 2])

# 将第二张观测图像设置为干净图像
# Cloud[:, :, :, 1] = 0
# Mask[:, :, :, 1] = 1
# Noisy[:, :, :, 1] = Clean[:, :, :, 1]
# G = Clean[:, :, :, 1].permute([2, 0, 1]).reshape(1, B, M, N).to(device)

################################ 网络输入 ################################

################################ 全局设置 ################################

def get_args(args_list):
    count = 0
    for i in range(len(args_list[0])):
        for j in range(len(args_list[1])):
            for k in range(len(args_list[2])):
                for l in range(len(args_list[3])):
                    # if not (i == j and j == k):
                    yield args_list[0][i], args_list[1][j], args_list[2][k], args_list[3][l]
                count += 1
# def grid_combine(args_list):
#     for rate in args_list[0]:
#         for rho in args_list[1]:
#             for lambda2 in args_list[2]:
#                 # if ((lambda1 / rho) < 1000) and ((lambda1 / rho) > 1):
#                     yield rate, rho, lambda2
def grid_combine(args_list):
    for lambda1 in args_list[0]:
        for lambda2 in args_list[1]:
            for rho in args_list[2]:
                    yield lambda1, lambda2, rho
args_list = [
    [25],  # rate
    [0.001, 0.01, 0.1, 1],  # rho
    [0.001, 0.01, 0.1, 1],  # lambda2
]
condidates = [
    # lambda1, lambda2, rho
    # [75, 0.02, 0.08],  # candidate5
    [45, 0.5, 0.1],    # candidate6
    [40, 0.2, 0.1],  # candidate7
    [35, 0.2, 0.1],   # candidate8
    [30, 0.2, 0.1],   # candidate8
    [40, 0.5, 0.1],    # candidate6
    [35, 0.5, 0.1],    # candidate6
    [30, 0.5, 0.1],    # candidate6
]
iter_num, epoch_num = 200, 300
order = 1
decrease = False
up = True
for rate, lambda2, rho in condidates:
    # rate = lambda1 / rho
    # rho_init = rho
    # lambda1 = rate * rho
    # lambda1, lambda2, rho, rate = 5.0, 0.01, 0.1, 50    # candidate1
    # lambda1, lambda2, rho, rate = 0.25, 0.001, 0.01, 25    # candidate3
    # lambda1, lambda2, rho, rate = 0.25, 0.01, 0.01, 25      # candidate4
    # lambda1, lambda2, rho, rate = 0.25, 0.1, 0.01, 25      # candidate5
    # rate, lambda2, rho = 75, 0.1, 0.08    # candidate6
    # rate, lambda2, rho, alpha = 20, 0.4, 0.01, 1.03
    alpha = 1.03
    lambda1 = rate * rho
    # rate = lambda1 / rho
    iters = 0
    order += 1
    log_dir = "./log-sentinel2-W/order[%03d]-iter[%3d]-epoch[%3d]-rate[%.3f]-lambda2[%.3f]-rho[%.3f]" \
              % (order, iter_num, epoch_num, rate, lambda2, rho)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    else:
    # print("log文件夹冲突！！！")
    # exit()
        continue

    ################################ 初始化 ################################
    Y = Noisy.clone()
    X = Noisy.clone()
    W = Noisy.clone()
    C = torch.zeros(Y.shape).type(dtype).to(device)
    model = MST(dim=16, stage=2, num_blocks=[2, 2, 2]).cuda()
    psnr = 0
    ################################ 子问题更新 ################################
    while iters < iter_num:
        iters += 1
        ################################ 更新子问题X ################################
        X = (torch.mul(Mask, (Y - C)) + rho * W ) / (Mask + rho)
        ################################ 更新子问题W ################################
        LR = 0.0002
        optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
        loss_history, loss1_history, loss2_history = [], [], []
        epoch = 0
        net_input = torch.permute(X, [2, 3, 0, 1]).reshape(1, B * T, M, N).type(dtype).to(device)
        while epoch < epoch_num:
            epoch += 1
            optimizer.zero_grad()
            # X: M, N, B, T
            Mask_temp = torch.permute(Mask, [2, 3, 1, 0]).reshape(1, B*T, M, N).type(dtype).to(device)
            out = model(net_input, Mask_temp)
            W = torch.permute(out.reshape(B, T, M, N), [2, 3, 0, 1])    # M, N, B, T
            loss1 = lambda1 * torch.norm(W.reshape(M*N, B*T), 'nuc')
            # loss1 = 1 / 2 * torch.norm(Y - torch.mul(X, Mask) - C, 'fro') ** 2
            loss2 = rho / 2 * torch.norm(X - W, 'fro') ** 2
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            # scheduler.step()
            loss_history.append(loss.item())
            loss1_history.append(loss1.item())
            loss2_history.append(loss2.item())
            if epoch % 40 == 0:
                print("\tThe %03dth iters, the %03dth epoch, loss: %.5f" % (iters, epoch, loss.item()))
            Y, C, W, X = Y.detach(), C.detach(), W.detach(), X.detach()

        Y, C, W, X = Y.detach(), C.detach(), W.detach(), X.detach()
        rho *= alpha

        ################################ 更新子问题W ################################
        # U, s, VH = torch.linalg.svd(X.reshape(M * N, B * T), full_matrices=False)  # B*T, M*N
        # s0 = s[0]
        # print("s: ", s)
        # s = s - lambda1 / rho
        # s[s < 0] = 0
        # S = torch.diag(s)
        # W = torch.mm(torch.mm(U, S), VH).reshape(M, N, B, T)
        ################################ 更新子问题C ################################
        L = Y - torch.mul(X, Mask)
        for t in range(T):
            C[:, :, :, t] = thres_21(L[:, :, :, t], lambda2, M, N, B)

        mse = torch.sum((X - Clean) ** 2)
        Recover = torch.mul(Y, Mask) + torch.mul(X, 1 - Mask)
        psnr = []
        for t in range(T):
            psnr_t = 0
            for b in range(B):
                psnr_t += compare_psnr(Recover[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy(), data_range=1)
            psnr.append(psnr_t/B)
        # psnr = compare_psnr(X.detach().cpu().numpy(), Clean.detach().cpu().numpy(), data_range=2)
        psnr1, psnr2, psnr3, psnr4 = psnr
        # ssim = []
        # for t in range(3):
        #     ssim_t = 0
        #     for b in range(B):
        #         ssim_t += compare_ssim(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy())
        #     ssim.append(ssim_t / B)
        # ssim1, ssim2, ssim3 = ssim
        
        print("The %03dth iters, the %03dth epoch, psnr[%.3f %.3f %.3f, %.3f]" % (
            iters, epoch, psnr1, psnr2, psnr3, psnr4))

        image_Clean = Clean.cpu().numpy() * 2
        image_Y = Y.cpu().numpy() * 2
        image_X = X.cpu().detach().numpy() * 2
        image_C = C.cpu().numpy() * 2
        plt.figure(figsize=(20, 20))
        for i in range(4):
            plt.subplot(5, 4, 1 + i)
            plt.title("Clean")
            plt.imshow(image_Clean[:, :, [3, 1, 0], i])
            plt.axis('off')

            plt.subplot(5, 4, 5 + i)
            plt.title("X")
            plt.imshow(image_X[:, :, [3, 1, 0], i])
            plt.axis('off')

            plt.subplot(5, 4, 9 + i)
            plt.title("Y")
            plt.imshow(image_Y[:, :, [3, 1, 0], i])
            plt.axis('off')

            plt.subplot(5, 4, 13 + i)
            plt.title("Y-X")
            plt.imshow(image_Y[:, :, [3, 1, 0], i] - image_X[:, :, [3, 1, 0], i])
            plt.axis('off')

            plt.subplot(5, 4, 17 + i)
            plt.title("C")
            plt.imshow(image_C[:, :, 0, i], cmap='gray')
            plt.axis('off')
        result_path = "%s/iter[%03d, %03d]-pnsr[%.3f, %.3f, %.3f, %.3f]-loss[%.5E].png" % (
            log_dir, iters, epoch, psnr1, psnr2, psnr3, psnr4, loss.item())
        plt.savefig(result_path)
        plt.clf()
        plt.figure(figsize=(30, 60))
        plt.subplot(6, 1, 1)
        plt.title("total loss")
        plt.plot([i for i in range(len(loss_history[50:]))], loss_history[50:])

        plt.subplot(6, 1, 2)
        plt.title("loss1")
        plt.plot([i for i in range(len(loss1_history[50:]))], loss1_history[50:])

        plt.subplot(6, 1, 3)
        plt.title("loss2")
        plt.plot([i for i in range(len(loss2_history[50:]))], loss2_history[50:])

        plt.subplot(6, 1, 4)
        plt.title("total loss")
        plt.plot([i for i in range(len(loss_history))], loss_history)

        plt.subplot(6, 1, 5)
        plt.title("loss1")
        plt.plot([i for i in range(len(loss1_history))], loss1_history)

        plt.subplot(6, 1, 6)
        plt.title("loss2")
        plt.plot([i for i in range(len(loss2_history))], loss2_history)

        loss_path = "%s/iter[%03d, %03d]-LR[%f]-rate[%.3f]-rho[%.3f].png" % (
            log_dir, iters, epoch, LR, lambda1 / rho, rho)
        plt.savefig(loss_path)
        plt.clf()

        plt.close()

    # 保存本次实验的恢复图像
    plt.close()
    scio.savemat("%s/recover_data.mat" % log_dir, {"recover_data": image_X})


	The 001th iters, the 040th epoch, loss: 5475.83931
	The 001th iters, the 080th epoch, loss: 5370.06349
	The 001th iters, the 120th epoch, loss: 5316.60415
	The 001th iters, the 160th epoch, loss: 5273.52543
	The 001th iters, the 200th epoch, loss: 5233.16736
	The 001th iters, the 240th epoch, loss: 5203.70925
	The 001th iters, the 280th epoch, loss: 5184.43299


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


The 001th iters, the 300th epoch, psnr[9.330 12.178 10.152, 8.448]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


	The 002th iters, the 040th epoch, loss: 4672.91921
	The 002th iters, the 080th epoch, loss: 4631.32527
	The 002th iters, the 120th epoch, loss: 4604.19803
	The 002th iters, the 160th epoch, loss: 4587.18173
	The 002th iters, the 200th epoch, loss: 4572.72338
	The 002th iters, the 240th epoch, loss: 4561.94848
	The 002th iters, the 280th epoch, loss: 4552.00215


...

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

In [ ]:
scio.savemat("%s/0recover_data.mat" % log_dir, {"recover_data" : image_X})
a = scio.loadmat("%s/0recover_data.mat" % log_dir)["recover_data"]
# scio.savemat("%s/recover_data.mat" % log_dir, {"recover_data", X.detach().cpu().numpy()})

In [ ]:
b = 39.56+45.52+39.35+38.35
b, b/4

In [ ]:
for i in range(4):
    psnr = compare_psnr(X[:, :, :, i].detach().cpu().numpy(), Clean[:, :, :, i].detach().cpu().numpy(), data_range=2)
    print(psnr)